In [5]:
%%capture
!pip install flickrapi

# Scrape links from flickr

In [6]:
import flickrapi
import requests
import pandas as pd

In [7]:
import flickr_credentials

api_key = flickr_credentials.KEY
api_secret = flickr_credentials.SECRET

In [8]:
URL_FORMAT = 'https://farm{farm_id}.staticflickr.com/{server_id}/{id}_{secret}.jpg'

In [9]:
flickr = flickrapi.FlickrAPI(api_key, api_secret, format='etree')

In [10]:
photos = flickr.walk(
    tag_mode='all',
    tags='azulejo,tile'
)

links = []

for photo in photos:
    link = URL_FORMAT.format(
        farm_id = photo.get('farm'),
        server_id = photo.get('server'),
        id=photo.get('id'),
        secret=photo.get('secret')
    )
    links.append(link)   

In [42]:
pd.Series(links).to_csv('links.csv', index=False, header=False)

# Downloading files locally

In [36]:
import csv
import re
import requests
import logging
import concurrent.futures

In [37]:
logger = logging.getLogger()
logger.setLevel('INFO')

In [38]:
with open('links.csv', 'r') as f:
    reader = csv.reader(f)
    links = [_[0] for _ in list(reader)]

In [49]:
links = links[1:]

In [63]:
def save_link(link):
    name = re.findall('.*/(.*).jpg', link)[0]
    try:
        with open('data/raw_images/{name}.jpg'.format(name=name), 'wb') as f:
            f.write(requests.get(link).content)
        logger.info('Downloaded {link}'.format(link=link))
    except Exception as e:
        logger.error(e)

In [64]:
save_link(links[0])

In [65]:
%%time

with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    [
        executor.submit(save_link, link)
        for link in links
    ]

CPU times: user 54 s, sys: 10.2 s, total: 1min 4s
Wall time: 24.1 s
